Reference:https://github.com/suragnair/alpha-zero-general/tree/master/othello

## Importing the libary

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from keras.optimizers import Adam
import random
import os
import math
import copy
import tensorflow as tf
import numpy as np
import random
import copy
import pickle
import re
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import keras.optimizers
tf.keras.utils.disable_interactive_logging()

## Board environment of "6x6 othello" game

In [17]:

class Board:
    def __init__(self, player1, player2, n=6):
        self.n = n
        self.state_size = n * n
        self.action_size = n * n
        self.board = [[0] * n for _ in range(n)]
        self.current_play = 1
        self.reset()
        self.player1 = player1
        self.player2 = player2
        self.done = False
        self.directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]

    def reset(self):
        n = self.n
        self.board = [[0] * n for _ in range(n)]
        self.current_play = 1
        if n % 2 == 0:
            mid = n // 2
            self.board[mid - 1][mid - 1] = 1
            self.board[mid][mid] = 1
            self.board[mid - 1][mid] = -1
            self.board[mid][mid- 1] = -1

    def get_board_state(self):
        return self.board

    def count_diff(self, color):
        count = 0
        for j in range(self.n):
            for i in range(self.n):
                if self.board[i][j] == color:
                    count += 1
                if self.board[i][j] == -color:
                    count -= 1
        return count

    def get_legal_moves(self, color):
        moves = set()
        for j in range(self.n):
            for i in range(self.n):
                if self.board[i][j] == color:
                    new_moves = self.get_moves_for_square((i, j))
                    moves.update(new_moves)
        return list(moves)

    def has_legal_moves(self, color):
        for j in range(self.n):
            for i in range(self.n):
                if self.board[i][j] == color:
                    new_moves = self.get_moves_for_square((i, j))
                    if len(new_moves) > 0:
                        return True
        return False

    def get_moves_for_square(self, square):
        (i, j) = square
        color = self.board[i][j]
        if color == 0:
            return None
        moves = []
        for direction in self.directions:
            move = self._discover_move(square, direction)
            if move:
                moves.append(move)
        return moves

    def execute_move(self, move, color):
        flips = []
        for direction in self.directions:
            flip = self._get_flips(move, direction, color)
            flips.extend(flip)

        if flips:
            for i, j in flips:
                self.board[i][j] = color
        else:
            raise Exception("Invalid move")
        row, col = move
        self.board[row][col] = color
        self.current_play = -1 if self.current_play == 1 else 1

    def _discover_move(self, origin, direction):
        i, j = origin
        color = self.board[i][j]
        flips = []
        for new_i, new_j in self._increment_move(origin, direction, self.n):
            if self.board[new_i][new_j] == 0:
                if flips:
                    return (new_i, new_j)
                else:
                    return None
            elif self.board[new_i][new_j] == color:
                return None
            elif self.board[new_i][new_j] == -color:
                flips.append((new_i, new_j))
        return None

    def _get_flips(self, origin, direction, color):
        flips = []
        for i, j in self._increment_move(origin, direction, self.n):
            if self.board[i][j] == 0:
                return []
            if self.board[i][j] == -color:
                flips.append((i, j))
            elif self.board[i][j] == color and len(flips) > 0:
                return flips
        return []

    def _increment_move(self, move, direction, n):
        move = tuple(move)[:2]  # Convert to tuple if it's a list
        i, j = move
        move = (i + direction[0], j + direction[1])
        while 0 <= move[0] < n and 0 <= move[1] < n:
            yield move
            move = (move[0] + direction[0], move[1] + direction[1])

    def get_reward(self):
        player1_score = self.count_diff(1)
        player2_score = self.count_diff(-1)
        if player1_score > player2_score:
            return 1
            
        elif player1_score < player2_score:
            self.player1.feedReward(-1)
            self.player2.feedReward(1)
            return -1
            
        else:
            self.player1.feedReward(0)
            self.player2.feedReward(0)
            return 0


    def game_over(self):
        return not self.has_legal_moves(1) and not self.has_legal_moves(-1)
    #train
    def play(self, rounds=200,inner_rounds=10):
        results = []
        for i in range(rounds):
            self.reset()
            current_player = self.player1 if self.current_play == 1 else self.player2
            if i % 100 == 0:
                print("Rounds {}".format(i))
            if i % 10 == 0:
                self.player1.setEps(rounds, i)
                self.player2.setEps(rounds, i)
            while True:
#                 self.print_board()
                positions = self.has_legal_moves(current_player.color)
                if positions:
                    p_move = current_player.choose_action(self)
                    self.execute_move(p_move, current_player.color)
                    current_player.addState(self.get_board_state())
                    state_after = copy.deepcopy(self.get_board_state())
                    current_player.addState(state_after)
                    if self.game_over():
#                         self.print_board()
                        winner = self.get_reward()
                        self.player1.feedReward(winner)
                        self.player2.feedReward(-winner)
                        reseults.append(winner)

                        self.player1.reset()
                        self.player2.reset()
                        self.reset()
                        break
                    current_player = self.player2 if current_player == self.player1 else self.player1
                else:
                    current_player = self.player2 if current_player == self.player1 else self.player1
                    if not (self.has_legal_moves(self.player1.color) or self.has_legal_moves(self.player2.color)):
                        self.player1.feedReward(0)
                        self.player2.feedReward(0)
                        results.append(0)
                        self.reset()
                        break
            
        return results
    #test
    def play2(self, rounds):
        for i in range(rounds):
            self.reset()
            current_player = self.player1 if self.current_play == 1 else self.player2
            while True:
                positions = self.has_legal_moves(current_player.color)
                if positions:
                    p_move = current_player.choose_action(self)
                    self.execute_move(p_move, current_player.color)
                    if self.game_over():
                        winner = self.get_reward()
                        if winner == 1:
                            return 1
                        elif winner == -1:
                            return -1
                        else:
                            return 0
                        break
                    current_player = self.player2 if current_player == self.player1 else self.player1
                else:
                    current_player = self.player2 if current_player == self.player1 else self.player1
                    if not (self.has_legal_moves(self.player1.color) or self.has_legal_moves(self.player2.color)):
                        break
        return

    def NNPlay(self, rounds=500, inner_rounds=10,name_exp= None):
        '''
        train inner rounds, capture replay buffer and train the s value networks on this data
        after every inner round the replay buffer is emptied and the process repeated again
        '''
        train_batches = int(rounds/inner_rounds) + 1
        for i in range(train_batches):
            self.play(inner_rounds)
            self.player1.sVNNtrain()
            self.player2.sVNNtrain()

        
    def print_board(self):
        for i in range(self.n):
            print('.........................')
            print('|', end=' ')
            for j in range(self.n):
                if self.board[i][j] == 1:
                    print('B', end=' ')
                elif self.board[i][j] == -1:
                    print('W', end=' ')
                else:
                    print(' ', end=' ')
                print('|', end=' ')
            print()
        print('...........................')



## DQN agent class

In [18]:

class DQNAgent:

    def __init__(self, color, n=6, eps_decay=False, start_exp_rate=0.2, end_exp_rate=0.02):
        self.n = n
        self.color = color
        self.states = []
        self.lr = 1.0
        self.exp_rate = start_exp_rate
        self.decay_gamma = 0.9
        self.states_value = {}
        self.eps_decay = eps_decay
        self.start_exp_rate = start_exp_rate
        self.end_exp_rate = end_exp_rate
        self.state_value_model = self.neural_network()

    def neural_network(self):
        model = keras.models.Sequential()
        model.add(Dense(units=36, input_dim=self.n * self.n, activation='relu'))
        model.add(Dense(18, activation='relu'))
        model.add(Dense(9, activation='relu'))
        model.add(Dense(1, activation='linear'))
        optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
        model.compile(loss='mean_squared_error', optimizer=optimizer)
        return model

    def get_hash(self, board):
        state_old = board.get_board_state()

        return np.array(state_old).flatten()

    def sNvalue(self, board):
        state_val = self.get_hash(board)
        state_val = state_val.reshape((1, self.n * self.n))
        pred = self.state_value_model.predict(state_val)[0]
        return pred

    def savemodel(self,path):
        self.state_value_model.save(path)

    def loadSVNNmodel(self, path):

        self.state_value_model = keras.models.load_model(path)

    def setEps(self, total_games, current_game):
        if self.eps_decay == False:
            return
        else:
            self.exp_rate = self.start_exp_rate * (1. - current_game/total_games) + self.end_exp_rate * (current_game/total_games)
            if np.mod(current_game, 100) == 0:
                print('decay rate modified at {} with current value of {}'.format(str(current_game), str(self.exp_rate)))
        return

    def choose_action(self, board):
        legal_moves = board.get_legal_moves(self.color)
        if np.random.uniform(0,1) < self.exp_rate:
            action_index = np.random.choice(len(legal_moves))
            selected_action = legal_moves[action_index]
            return selected_action
        else:
            value_max = -999
            best_action = None
            for p in legal_moves:
                next_board = copy.deepcopy(board)
                next_board.execute_move(p, self.color)
                next_state_hash = self.get_hash(next_board)
                pred = self.sNvalue(next_board)
                if pred >= value_max:
                    value_max = pred
                    best_action = p
            return best_action

    def getbuffer(self):
        data = self.states_value
        ll = []
        for k in data.keys():
            k_str = str(k)
            yy = re.findall(r'[-/+]?\d+\.*\d*', k_str)
            zz = data.get(k)
            yy.append(zz)
            ll.append(yy)
        lldf = pd.DataFrame(ll)
        cols = ['x'+str(i) for i in range(self.n*self.n)]
        cols.append('val')
        lldf.columns = cols
        lldf.to_csv('Buffer1.csv', index=False)
        return

    def sVNNtrain(self, Xin=None, Yin=None):
        self.getbuffer()
        df = pd.read_csv('Buffer1.csv')
        print ('length of buffer is', len(df))
        traincols=['x' + str(i) for i in range(self.n*self.n)]
        testcol = 'val'
        if Xin is None:
            Xin = df[traincols]

            Yin = df[testcol]
        self.state_value_model.fit(Xin, Yin, epochs=10, verbose=False)
        self.states_value = {}
        return
    def addState(self, state):
        state_tuple = tuple(map(tuple, state))
        self.states.append(state_tuple)

    def feedReward(self, reward):
        for st in reversed(self.states):
            if self.states_value.get(st) is None:
                self.states_value[st] = 0
            self.states_value[st] += self.lr * (self.decay_gamma * reward - self.states_value[st])
            reward = self.states_value[st]



    def reset(self):
        self.states = []


## Random player

In [19]:
class RandomPlayer:
    def __init__(self, color, name):
        self.color = color
        self.name = name

    def choose_action(self, board):
        legal_moves = board.get_legal_moves(self.color)
        if legal_moves:
            return random.choice(legal_moves)
        return None

    def addState(self, state, action, reward, next_state, done):
        pass

    def feedReward(self, minibatch):
        pass

    def reset(self):
        pass


## Pre trained both are computer player

In [20]:
player1 = DQNAgent(1,n=6,eps_decay=False, start_exp_rate=0.2, end_exp_rate=0.02)
player2 = DQNAgent(-1,n=6,eps_decay=False, start_exp_rate=0.2, end_exp_rate=0.02)

print(player1.exp_rate,player2.exp_rate)

0.2 0.2


In [21]:
player1.state_value_model.summary()

In [22]:
mm = player1.state_value_model
for layer in mm.layers: print(layer.get_config(), layer.get_weights())

{'name': 'dense_8', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, 36), 'units': 36, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None} [array([[-0.11438002,  0.18225381,  0.22303039, ...,  0.11660501,
         0.14002013,  0.00962228],
       [ 0.15687421,  0.00655755, -0.0084734 , ...,  0.07064864,
         0.11626178, -0.14710903],
       [ 0.18676004,  0.2726534 ,  0.1711398 , ...,  0.07000884,
         0.22672093, -0.12673867],
       ...,
       [-0.26042253, -0.01448852, -0.0343385 , ...,  0.1461649 ,
        -0.164464  ,  0.05871609],
       [ 0.04448387, -0.24791631,  0.22108573, ..., 

In [23]:
mm = player2.state_value_model
for layer in mm.layers: print(layer.get_config(), layer.get_weights())

{'name': 'dense_12', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, 36), 'units': 36, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None} [array([[ 0.05469212, -0.24265508, -0.11037017, ...,  0.01560152,
         0.10469303, -0.07856192],
       [-0.20590323,  0.02267972, -0.10246068, ..., -0.00935262,
         0.05296379, -0.22276989],
       [-0.24398452, -0.04477058, -0.1759165 , ..., -0.09464678,
        -0.11692567,  0.13913986],
       ...,
       [ 0.08306646,  0.18371141,  0.15009758, ...,  0.18283719,
        -0.07176444, -0.01019946],
       [-0.2143563 ,  0.11770636,  0.2820741 , ...,

In [25]:
board=Board(player1,player2,n=6)
rounds = 1000
for i in range(10):
    result = board.play(rounds)
    print("Round {} in the batch of 10 games: Player 1 wins {}, Player 2 wins {}, Draws {}".format(
        i,
        result.count(1),
        result.count(-1),
        result.count(0)
    ))


Rounds 0
Round 0 in the batch of 10 games: Player 1 wins 7, Player 2 wins 3, Draws 0
Rounds 0
Round 1 in the batch of 10 games: Player 1 wins 7, Player 2 wins 2, Draws 1
Rounds 0
Round 2 in the batch of 10 games: Player 1 wins 6, Player 2 wins 3, Draws 1
Rounds 0
Round 3 in the batch of 10 games: Player 1 wins 5, Player 2 wins 4, Draws 1
Rounds 0
Round 4 in the batch of 10 games: Player 1 wins 6, Player 2 wins 4, Draws 0
Rounds 0
Round 5 in the batch of 10 games: Player 1 wins 7, Player 2 wins 3, Draws 0
Rounds 0
Round 6 in the batch of 10 games: Player 1 wins 7, Player 2 wins 3, Draws 0
Rounds 0
Round 7 in the batch of 10 games: Player 1 wins 5, Player 2 wins 5, Draws 0
Rounds 0
Round 8 in the batch of 10 games: Player 1 wins 6, Player 2 wins 4, Draws 0
Rounds 0
Round 9 in the batch of 10 games: Player 1 wins 7, Player 2 wins 3, Draws 0


## First player random play against computer

In [29]:
player5 = DQNAgent(1, n=6, eps_decay=False, start_exp_rate=1, end_exp_rate=0)

player6 = DQNAgent(-1, n=6, eps_decay=False, start_exp_rate=0, end_exp_rate=0)


In [30]:
board = Board(player5, player6, n=6)
for i in range(10):
    result = board.play(10)
    print("Round {} in the batch of 10 games: Player 1 wins {}, Player 2 wins {}, Draws {}".format(
        i,
        result.count(1),
        result.count(-1),
        result.count(0)
    ))


Rounds 0
Round 0 in the batch of 10 games: Player 1 wins 5, Player 2 wins 5, Draws 0
Rounds 0
Round 1 in the batch of 10 games: Player 1 wins 7, Player 2 wins 1, Draws 2
Rounds 0
Round 2 in the batch of 10 games: Player 1 wins 6, Player 2 wins 4, Draws 0
Rounds 0
Round 3 in the batch of 10 games: Player 1 wins 4, Player 2 wins 6, Draws 0
Rounds 0
Round 4 in the batch of 10 games: Player 1 wins 4, Player 2 wins 6, Draws 0
Rounds 0
Round 5 in the batch of 10 games: Player 1 wins 2, Player 2 wins 8, Draws 0
Rounds 0
Round 6 in the batch of 10 games: Player 1 wins 5, Player 2 wins 4, Draws 1
Rounds 0
Round 7 in the batch of 10 games: Player 1 wins 7, Player 2 wins 3, Draws 0
Rounds 0
Round 8 in the batch of 10 games: Player 1 wins 6, Player 2 wins 3, Draws 1
Rounds 0
Round 9 in the batch of 10 games: Player 1 wins 3, Player 2 wins 6, Draws 1


## Train RL agent uaing Neural Network

In [31]:
board.NNPlay(200)

Rounds 0
length of buffer is 1500
length of buffer is 1465
Rounds 0
length of buffer is 149
length of buffer is 151
Rounds 0
length of buffer is 151
length of buffer is 149
Rounds 0
length of buffer is 153
length of buffer is 148
Rounds 0
length of buffer is 159
length of buffer is 143
Rounds 0
length of buffer is 155
length of buffer is 149
Rounds 0
length of buffer is 154
length of buffer is 150
Rounds 0
length of buffer is 150
length of buffer is 156
Rounds 0
length of buffer is 150
length of buffer is 155
Rounds 0
length of buffer is 151
length of buffer is 148
Rounds 0
length of buffer is 150
length of buffer is 151
Rounds 0
length of buffer is 147
length of buffer is 152
Rounds 0
length of buffer is 152
length of buffer is 148
Rounds 0
length of buffer is 150
length of buffer is 148
Rounds 0
length of buffer is 150
length of buffer is 152
Rounds 0
length of buffer is 151
length of buffer is 151
Rounds 0
length of buffer is 152
length of buffer is 147
Rounds 0
length of buffer is 

In [32]:
mm = board.player1.state_value_model
for layer in mm.layers: print(layer.get_config(), layer.get_weights())

{'name': 'dense_32', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, 36), 'units': 36, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None} [array([[-0.28329733, -0.0602001 ,  0.3734203 , ...,  0.2965747 ,
        -0.24881937,  0.13013777],
       [-0.07047888, -0.05094478,  0.15261325, ..., -0.26295835,
        -0.33232462,  0.01716913],
       [ 0.12891634,  0.12713924,  0.17539693, ...,  0.34828705,
        -0.31632057, -0.29993108],
       ...,
       [ 0.00055409, -0.20427752,  0.2839142 , ..., -0.1993362 ,
        -0.22015429, -0.27926368],
       [ 0.14020835, -0.26250514, -0.0306044 , ...,

In [33]:
mm = board.player2.state_value_model
for layer in mm.layers: print(layer.get_config(), layer.get_weights())

{'name': 'dense_36', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, 36), 'units': 36, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None} [array([[ 0.30677456, -0.0779414 ,  0.09485174, ...,  0.03167492,
         0.20160279, -0.02455483],
       [ 0.16407216, -0.24243316, -0.35013384, ...,  0.18980193,
        -0.10024292, -0.26496467],
       [-0.09801096, -0.00502521,  0.02440263, ...,  0.03162536,
         0.33909056, -0.08403892],
       ...,
       [ 0.25067642, -0.21480577, -0.05440513, ...,  0.00433526,
        -0.07836007,  0.07435226],
       [ 0.19760312,  0.27661416,  0.01794398, ...,

## Save their model

In [34]:
board.player1.savemodel("model_p1.keras")
board.player2.savemodel("model_p2.keras")

## First player is computer and second is random

In [35]:
player3 = DQNAgent(1,n=6, eps_decay = False, start_exp_rate=0, end_exp_rate=0)

In [36]:
player3.loadSVNNmodel("model_p1.keras")

In [37]:
dd = player3.state_value_model
for layer in dd.layers: print(layer.get_config(), layer.get_weights())

{'name': 'dense_32', 'trainable': True, 'dtype': 'float32', 'batch_input_shape': (None, 36), 'units': 36, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None} [array([[-0.28329733, -0.0602001 ,  0.3734203 , ...,  0.2965747 ,
        -0.24881937,  0.13013777],
       [-0.07047888, -0.05094478,  0.15261325, ..., -0.26295835,
        -0.33232462,  0.01716913],
       [ 0.12891634,  0.12713924,  0.17539693, ...,  0.34828705,
        -0.31632057, -0.29993108],
       ...,
       [ 0.00055409, -0.20427752,  0.2839142 , ..., -0.1993362 ,
        -0.22015429, -0.27926368],
       [ 0.14020835, -0.26250514, -0.0306044 , ...,

In [38]:
player3 = DQNAgent(1,n=6, eps_decay = False, start_exp_rate=0, end_exp_rate=0)
player3.loadSVNNmodel("model_p1.keras")
player4=DQNAgent(-1,n=6,eps_decay=False,start_exp_rate=1,end_exp_rate=0)

In [39]:
board = Board(player3, player4, n=6)

In [40]:
for i in range(10):
    result = board.play(10)
    print("Round {} in the batch of 10 games: Player 1 wins {}, Player 2 wins {}, Draws {}".format(
        i,
        result.count(1),
        result.count(-1),
        result.count(0)
    ))

Rounds 0
Round 0 in the batch of 10 games: Player 1 wins 9, Player 2 wins 1, Draws 0
Rounds 0
Round 1 in the batch of 10 games: Player 1 wins 4, Player 2 wins 5, Draws 1
Rounds 0
Round 2 in the batch of 10 games: Player 1 wins 8, Player 2 wins 2, Draws 0
Rounds 0
Round 3 in the batch of 10 games: Player 1 wins 7, Player 2 wins 3, Draws 0
Rounds 0
Round 4 in the batch of 10 games: Player 1 wins 5, Player 2 wins 4, Draws 1
Rounds 0
Round 5 in the batch of 10 games: Player 1 wins 5, Player 2 wins 4, Draws 1
Rounds 0
Round 6 in the batch of 10 games: Player 1 wins 6, Player 2 wins 4, Draws 0
Rounds 0
Round 7 in the batch of 10 games: Player 1 wins 7, Player 2 wins 2, Draws 1
Rounds 0
Round 8 in the batch of 10 games: Player 1 wins 7, Player 2 wins 3, Draws 0
Rounds 0
Round 9 in the batch of 10 games: Player 1 wins 5, Player 2 wins 5, Draws 0


## First player is Random and second is computer

In [43]:
player5 = DQNAgent(1, n=6, eps_decay=False, start_exp_rate=1, end_exp_rate=0)

player6 = DQNAgent(-1, n=6, eps_decay=False, start_exp_rate=0, end_exp_rate=0)
player5.loadSVNNmodel("model_p2.keras")

In [ ]:
board = Board(player5, player6, n=6)
for i in range(10):
    result = board.play(10)
    print("Round {} in the batch of 10 games: Player 1 wins {}, Player 2 wins {}, Draws {}".format(
        i,
        result.count(1),
        result.count(-1),
        result.count(0)
    ))


Rounds 0
Round 0 in the batch of 10 games: Player 1 wins 3, Player 2 wins 7, Draws 0
Rounds 0
Round 1 in the batch of 10 games: Player 1 wins 3, Player 2 wins 5, Draws 2
Rounds 0
Round 2 in the batch of 10 games: Player 1 wins 4, Player 2 wins 6, Draws 0
Rounds 0
Round 3 in the batch of 10 games: Player 1 wins 2, Player 2 wins 5, Draws 3
Rounds 0
Round 4 in the batch of 10 games: Player 1 wins 3, Player 2 wins 6, Draws 1
Rounds 0
Round 5 in the batch of 10 games: Player 1 wins 2, Player 2 wins 7, Draws 1
Rounds 0
Round 6 in the batch of 10 games: Player 1 wins 4, Player 2 wins 6, Draws 0
Rounds 0
Round 7 in the batch of 10 games: Player 1 wins 4, Player 2 wins 6, Draws 0
Rounds 0
Round 8 in the batch of 10 games: Player 1 wins 5, Player 2 wins 5, Draws 0
Rounds 0
